In [3]:
from google.colab import drive
drive.mount("/content/drive/",force_remount=True)

Mounted at /content/drive/


In [4]:
import math
import os
import imutils
import numpy as np
import pandas as pd
import cv2 as cv2
import glob
import csv
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from scipy import interp
from sklearn.multiclass import OneVsRestClassifier
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
from sklearn.svm import SVC
from skimage import io

In [5]:
def _process_row(row):
    """
    Process a single dataframe row, returns the argmax label
    :param row:
    :return:
    """
    return np.argmax(row)

In [7]:
label_file = os.path.join("/content/drive/MyDrive/FER/Data/Images/FER2013Train/", "label.csv")
data_df = pd.read_csv(label_file, header=None)
data_df.columns = ["img_name", "dims", "0", "1", "2", "3", "4", "5", "6", "7","Unknown", "NF"]
data_df['actual_label'] = data_df[['0', '1', '2', '3', '4', '5', '6', '7']].apply(lambda x: _process_row(x), axis=1)

# get all ilocs with actual label 0
data_df = data_df.sort_values(by=['img_name'])

image_file_names = data_df['img_name'].values

In [8]:
len(image_file_names)

28558

In [9]:
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

In [10]:
def returnLocalizedFace(img):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray =cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:

        roi_gray = gray[y:y + h, x:x + w]
        roi_color = img[y:y + h, x:x + w]

    if len(faces) == 0:
        return img
    crop_img = img[y:y + h, x:x + w]
    return crop_img


def getImage(path):
    return cv2.imread(path)
def show(img):
    cv2.imshow('im', img)
    cv2.waitKey(0)

In [11]:
X = []
y = []
mean = 0
var = 10
sigma = var ** 0.5
gaussian = np.random.normal(mean, sigma, (224, 224)) 

def read(imagesPath, data_df):
    i = 0
    for filename in glob.glob("/content/drive/MyDrive/FER/Data/Images/FER2013Train" + '/*.*'):
        i = i+1
        file = filename.split('/')[-1]
        print(file,i)
        if ((file != 'label.csv') & (file in image_file_names)):
          win_size = (64, 128)
          img = returnLocalizedFace(getImage(filename))

          win_size = (64, 128)

          img = cv2.resize(img, win_size)
          img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

          img1 = cv2.imread(filename)
          img1 = cv2.resize(img1, (224,224))
          noisy_image = np.zeros(img1.shape, np.float32)
          noisy_image[:, :, 0] = img1[:, :, 0] + gaussian
          noisy_image[:, :, 1] = img1[:, :, 1] + gaussian
          noisy_image[:, :, 2] = img1[:, :, 2] + gaussian
          cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
          noisy_image = noisy_image.astype(np.uint8)

          d = cv2.HOGDescriptor()
          hog = d.compute(img)
          X.append(hog.transpose()[0])
          y.append(data_df[data_df['img_name'] == file]['actual_label'].values[0])

          hog1 = d.compute(img1)
          X.append(hog1.transpose()[0])
          y.append(data_df[data_df['img_name'] == file]['actual_label'].values[0])

          # image_to_transform = sk.io.imread(filename)
          # transformed_image = random_rotation(image_to_transform)
          # hog2 = d.compute(transformed_image)
          # X.append(hog2.transpose()[0])
          # y.append(data_df[data_df['img_name'] == file]['actual_label'].values[0])         

def fromIndexToFeatures(X, indecies):
    features = []
    for i in indecies:
        features.append(X[i])
    return np.asarray(features)

def fromIndexToLabels(y, indecies):
    labels = []
    for i in indecies:
        labels.append(y[i])
    return np.asarray(labels)

In [ ]:
read("/content/drive/MyDrive/FER/Data/Images/FER2013Train",data_df)

fer0027955.png 1
fer0027961.png 2
fer0027268.png 3
fer0027523.png 4
fer0027594.png 5
fer0027568.png 6
fer0027753.png 7
fer0027154.png 8
fer0027780.png 9
fer0027263.png 10
fer0027273.png 11
fer0028146.png 12
fer0027657.png 13
fer0027200.png 14
fer0028104.png 15
fer0027082.png 16
fer0027381.png 17
fer0027975.png 18
fer0028171.png 19
fer0027428.png 20
fer0027896.png 21
fer0027474.png 22
fer0027389.png 23
fer0027266.png 24
fer0027117.png 25
fer0027478.png 26
fer0027972.png 27
fer0028405.png 28
fer0027982.png 29
fer0027040.png 30
fer0027264.png 31
fer0027382.png 32
fer0027790.png 33
fer0028169.png 34
fer0028293.png 35
fer0027709.png 36
fer0027506.png 37
fer0027345.png 38
fer0027801.png 39
fer0027807.png 40
fer0028443.png 41
fer0027614.png 42
fer0028175.png 43
fer0027803.png 44
fer0027880.png 45
fer0027190.png 46
fer0027941.png 47
fer0028415.png 48
fer0027460.png 49
fer0027969.png 50
fer0027707.png 51
fer0027983.png 52
fer0027219.png 53
fer0027834.png 54
fer0028244.png 55
fer0028327 (1).png 

In [ ]:
y = np.asarray(y)
X = np.asarray(X)